In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from keras.applications.vgg16 import VGG16
vgg_model=VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=2,classifier_activation='sigmoid')

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train=ImageDataGenerator(rescale=1./255,
                            rotation_range=5,
                            zoom_range=.2,
                            horizontal_flip=True,validation_split=0.2)
test=ImageDataGenerator(rescale=1./255)
train_data=train.flow_from_directory('/content/gdrive/MyDrive/dataset/datasetweek2/DogVsCat/training_set',color_mode='rgb',
                                        target_size=(224,224),
                                        batch_size=1,
                                        class_mode='binary',
                                        shuffle=True,subset="training")
val_data=train.flow_from_directory('/content/gdrive/MyDrive/dataset/datasetweek2/DogVsCat/training_set',color_mode='rgb',
                                        target_size=(224,224),
                                        batch_size=1,
                                        class_mode='binary',
                                        shuffle=True,subset="validation")
test_data=test.flow_from_directory('/content/gdrive/MyDrive/dataset/datasetweek2/DogVsCat/test_set',color_mode='rgb',
                                        target_size=(224,224),
                                        batch_size=1,
                                        class_mode='binary',
                                        shuffle=True)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [ ]:
for layer in vgg_model.layers:
    layer.trainable = False

In [ ]:
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
x = Flatten()(vgg_model.output)
x=Dense(4096,activation='relu')(x)

In [ ]:

prediction=Dense(1, activation='sigmoid')(x)


In [ ]:
model = Model(inputs=vgg_model.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
# fit the model
history = model.fit(
 train_data,
  
  validation_data=val_data,
  epochs=10,steps_per_epoch=len(train_data)//32,
  callbacks=[early_stop,checkpoint],
  batch_size=32,shuffle=True)

Epoch 1/10
200/200 [==============================] - ETA: 0s - loss: 5.9882 - accuracy: 0.6150

200/200 [==============================] - 56s 276ms/step - loss: 5.9882 - accuracy: 0.6150 - val_loss: 1.2447 - val_accuracy: 0.7708
Epoch 2/10
200/200 [==============================] - ETA: 0s - loss: 1.2978 - accuracy: 0.7550

200/200 [==============================] - 42s 210ms/step - loss: 1.2978 - accuracy: 0.7550 - val_loss: 1.3329 - val_accuracy: 0.6352
Epoch 3/10
199/200 [============================>.] - ETA: 0s - loss: 0.6513 - accuracy: 0.7387

200/200 [==============================] - 46s 230ms/step - loss: 0.6602 - accuracy: 0.7350 - val_loss: 0.5265 - val_accuracy: 0.7801
Epoch 4/10
199/200 [============================>.] - ETA: 0s - loss: 0.5681 - accuracy: 0.7990

200/200 [==============================] - 46s 228ms/step - loss: 0.5721 - accuracy: 0.7950 - val_loss: 0.3744 - val_accuracy: 0.8370
Epoch 5/10
199/200 [============================>.] - ETA: 0s - loss: 0.3709 - accuracy: 0.8543

...

200/200 [==============================] - 47s 235ms/step - loss: 0.3445 - accuracy: 0.8550 - val_loss: 0.2968 - val_accuracy: 0.8751
Epoch 7/10
199/200 [============================>.] - ETA: 0s - loss: 0.3307 - accuracy: 0.8442

200/200 [==============================] - 47s 234ms/step - loss: 0.3321 - accuracy: 0.8450 - val_loss: 0.2592 - val_accuracy: 0.8851
Epoch 8/10
199/200 [============================>.] - ETA: 0s - loss: 0.3127 - accuracy: 0.8794

200/200 [==============================] - 48s 242ms/step - loss: 0.3112 - accuracy: 0.8800 - val_loss: 0.3277 - val_accuracy: 0.8570
Epoch 9/10
199/200 [============================>.] - ETA: 0s - loss: 0.4598 - accuracy: 0.7889

200/200 [==============================] - 48s 241ms/step - loss: 0.4598 - accuracy: 0.7900 - val_loss: 0.3180 - val_accuracy: 0.8595
Epoch 10/10
199/200 [============================>.] - ETA: 0s - loss: 0.3231 - accuracy: 0.8492

200/200 [==============================] - 50s 250ms/step - loss: 0.3386 - accuracy: 0.8450 - val_loss: 0.2642 - val_accuracy: 0.8826


In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np
#predict
y_pred=model.predict(test_data)
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_data.classes,y_pred)

array([[1011,    0],
       [1012,    0]])

In [ ]:

model.save("/content/drive/MyDrive/VGG_dogsVscats.h5")
print("Saved model to disk")
model.save_weights("/content/drive/MyDrive/VGG_dogsVscats.h5")

Saved model to disk
